#데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/데이터마이닝 과제/플젝/preprocessed_data.csv')

In [ ]:
data.head()

,Unnamed: 0,label,neg_score,pos_socre,location_강원,location_경기,location_경남,location_경북,location_광주,location_대구,location_대전,location_부산,location_서울,location_세종,location_울산,location_인천,location_전남,location_전북,location_제주,location_충남,location_충북,species_골든 리트리버,species_그레이 하운드,species_그레이트 피레니즈,species_기타,species_닥스훈트,species_도베르만,species_동경견,species_라브라도 리트리버,species_로트바일러,species_마스티프,species_말라뮤트,species_말티즈,species_미니어쳐 푸들,species_미니어쳐 핀셔,species_믹스견,species_바센지,species_벨기에 그로넨달,species_보더 콜리,species_보스턴 테리어,...,species_슈나우져,species_스탠다드 푸들,species_스피츠,species_시바,species_시베리안 허스키,species_시츄,species_아메리칸 코카 스파니엘,species_아프간 하운드,species_올드 잉글리쉬 불독,species_올드 잉글리쉬 쉽독,species_요크셔 테리어,species_웰시 코기 카디건,species_웰시 코기 펨브로크,species_잉글리쉬 코카 스파니엘,species_재패니즈 스피츠,species_제주개,species_진도견,species_차우차우,species_치와와,species_케언 테리어,species_케인 코르소,species_코리아 트라이 하운드,species_코카 스파니엘,species_코카시안오브차카,species_토이 푸들,species_페키니즈,species_페터데일테리어,species_포메라니안,species_푸들,species_풍산견,species_프렌치 불독,species_핏불테리어,color_검,color_황,color_흰,sex_수컷,sex_암컷,neutralized_미상,neutralized_아니오,neutralized_예
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
3,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
4,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0


In [ ]:
data.info()

In [ ]:
data = data.drop('Unnamed: 0', axis=1)

In [ ]:
data.head()

,label,neg_score,pos_socre,location_강원,location_경기,location_경남,location_경북,location_광주,location_대구,location_대전,location_부산,location_서울,location_세종,location_울산,location_인천,location_전남,location_전북,location_제주,location_충남,location_충북,species_골든 리트리버,species_그레이 하운드,species_그레이트 피레니즈,species_기타,species_닥스훈트,species_도베르만,species_동경견,species_라브라도 리트리버,species_로트바일러,species_마스티프,species_말라뮤트,species_말티즈,species_미니어쳐 푸들,species_미니어쳐 핀셔,species_믹스견,species_바센지,species_벨기에 그로넨달,species_보더 콜리,species_보스턴 테리어,species_불독,...,species_슈나우져,species_스탠다드 푸들,species_스피츠,species_시바,species_시베리안 허스키,species_시츄,species_아메리칸 코카 스파니엘,species_아프간 하운드,species_올드 잉글리쉬 불독,species_올드 잉글리쉬 쉽독,species_요크셔 테리어,species_웰시 코기 카디건,species_웰시 코기 펨브로크,species_잉글리쉬 코카 스파니엘,species_재패니즈 스피츠,species_제주개,species_진도견,species_차우차우,species_치와와,species_케언 테리어,species_케인 코르소,species_코리아 트라이 하운드,species_코카 스파니엘,species_코카시안오브차카,species_토이 푸들,species_페키니즈,species_페터데일테리어,species_포메라니안,species_푸들,species_풍산견,species_프렌치 불독,species_핏불테리어,color_검,color_황,color_흰,sex_수컷,sex_암컷,neutralized_미상,neutralized_아니오,neutralized_예
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8359 entries, 0 to 8358
Data columns (total 86 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   label                 8359 non-null   int64
 1   neg_score             8359 non-null   int64
 2   pos_socre             8359 non-null   int64
 3   location_강원           8359 non-null   int64
 4   location_경기           8359 non-null   int64
 5   location_경남           8359 non-null   int64
 6   location_경북           8359 non-null   int64
 7   location_광주           8359 non-null   int64
 8   location_대구           8359 non-null   int64
 9   location_대전           8359 non-null   int64
 10  location_부산           8359 non-null   int64
 11  location_서울           8359 non-null   int64
 12  location_세종           8359 non-null   int64
 13  location_울산           8359 non-null   int64
 14  location_인천           8359 non-null   int64
 15  location_전남           8359 non-null   int64
 16  locati

#X, y split

In [ ]:
X = data.drop('label', axis=1)
y = data[['label']]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3, random_state=42)

#모델 생성(0.72)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = y_pred.reshape(-1, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.79      0.74      1296
           1       0.74      0.64      0.69      1212

    accuracy                           0.72      2508
   macro avg       0.72      0.71      0.71      2508
weighted avg       0.72      0.72      0.71      2508



#GridSearch (0.76)

In [ ]:
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [ ]:
from sklearn.model_selection import GridSearchCV

knn_parameters = [{
    'n_neighbors': [1, 3, 5, 7, 9, 11],
    'leaf_size': [5, 10, 15, 20],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'n_jobs': [-1],
    'weights': ['uniform', 'distance']}]

grid_model = GridSearchCV(model, param_grid = knn_parameters, cv=3, scoring = 'accuracy')

grid_model.fit(X_train, y_train.ravel())
y_pred_grid = grid_model.predict(X_test)

In [ ]:
print(grid_model.best_params_)

{'algorithm': 'ball_tree', 'leaf_size': 10, 'n_jobs': -1, 'n_neighbors': 11, 'weights': 'distance'}


In [ ]:
print(classification_report(y_test, y_pred_grid))

              precision    recall  f1-score   support

           0       0.74      0.84      0.79      1296
           1       0.80      0.68      0.73      1212

    accuracy                           0.76      2508
   macro avg       0.77      0.76      0.76      2508
weighted avg       0.77      0.76      0.76      2508



#Feature Selection - RFE

In [ ]:
X.shape

(8359, 85)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score

best_score = 0
best_feature = []

for i in range(1, 86) : 

  selector = RFE(grid_model, n_features_to_select=i)
  selector = selector.fit(X_train, y_train.ravel())

  feature_bool = selector.support_

  temp = []

  for j in range(len(feature_bool)):
    if feature_bool[j] == True:
      temp.append(j)

  X_train_selected = X_train.iloc[:, temp]
  X_test_selected = X_test.iloc[:, temp]

  grid_model.fit(X_train_selected, y_train)
  y_pred = grid_model.predict(X_test_selected)

  print(i)
  score = accuracy_score(y_pred, y_test)

  if best_score < score :
    best_score = score
    best_feature = temp

RuntimeError: ignored